## Load GranD DamList and Inflow time-series

In [2]:
import os
import numpy as np
import pandas as pd
from rpy2.robjects import r
from rpy2.robjects import pandas2ri
pandas2ri.activate()
import tools
import geopandas as gpd

### Create a Row-major order shapefile of the 0.5 degree global graticule
We change the cru_id of "cru_ts.shp" from Column-major order to Row-major order.


In [3]:
if False:
    old = gpd.read_file('./data/cru_ts.shp')
    subs = np.unravel_index(old.cru_id-1, [360,720], 'F')
    reidx = np.ravel_multi_index(subs, [360,720], order='C')
    new = old.copy()
    new['cru_id'] = reidx
    new = new.sort_values('cru_id').reset_index(drop=True)
    new.to_file('./data/cru_reindexed.shp') # Validated

### Generate DamList with global index and Inflow data from Rdata

In [4]:
# Read RData
rf = r['load']('./data/rdata/inflows_MCM_1958_20181126.Rdata')
rf = r['new_inflows_1958']
data = np.zeros([len(rf),516+1])
for i in range(len(rf)):
#     data[i,0] = pandas2ri.ri2py_intvector(rf[i][0])
#     data[i,1:] = pandas2ri.ri2py_intvector(rf[i][3])
    data[i,0] = rf[i][0]
    data[i,1:] = rf[i][3]

# Convert to DataFrame
damList = data[:,0].astype(int)
mondex = pd.period_range('{:04d}-{:02d}'.format(1958,1), periods=516, freq='M')
dfFlowDams = pd.DataFrame(data=data[:,1:].T, index=mondex, columns=damList)

In [6]:
# Load Latitude and Longitude for the selected GranD dams
fn = os.path.join('./data/grand1593_uparea.xlsx')
xl = pd.ExcelFile(fn)
df = xl.parse('cruidToLatLon')
gridIndx = df.cruid_aligned.values - 1

# Get upstream areas at the grids of interest
subs = np.unravel_index(gridIndx, [360,720], 'F')
ind_dams = np.ravel_multi_index(subs, [360,720], order='C')
ind_dams = np.vstack((df.grand_no, ind_dams))

# Reorder "ind_dams" to be matched with "damList"
order = np.array([np.argwhere(ind_dams[0,:] == did)[0][0] for did in damList])
ind_dams = ind_dams[:,order]
ind_dams

<ipython-input-6-007148f908e1>:3: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  xl = pd.ExcelFile(fn)


array([[    6,    24,    25, ...,  2078,  4795,  6521],
       [48355, 57708, 57708, ..., 65010, 85477, 79115]])

In [7]:
# Save Data (dfFlowDams, ind_dams)
if True:
    filn = './data/dfFlowDams.hdf'
#     dfFlowDams.to_hdf(filn, key='df', complib='blosc:zstd', complevel=9)
    dfFlowDams.to_hdf(filn, key='df')
    print('%s is saved.' % filn)
    filn = './data/ind_dams'
    np.savez_compressed(filn, ind_dams = ind_dams)
    print('%s.npz is saved.' % filn)

./data/dfFlowDams.hdf is saved.
./data/ind_dams.npz is saved.
